Load images from the specified folder into a list.

In [ ]:
import os
from PIL import Image

image_folder = '../input/cell_images/'
images = [Image.open(os.path.join(image_folder, f)) for f in os.listdir(image_folder) if f.endswith('.png')]

Define data transformations to preprocess the images.

In [ ]:
import torchvision.transforms as transforms

data_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

Create a custom dataset class to load images and apply transformations.

In [ ]:
from torch.utils.data import DataLoader, Dataset

class CellDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image

cell_dataset = CellDataset(images, transform=data_transform)

Initialize DataLoaders for training and validation datasets.

In [ ]:
train_loader = DataLoader(cell_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(cell_dataset, batch_size=32, shuffle=False)

Define a simple neural network architecture and the loss function.

In [ ]:
import torch.nn as nn
import torch.optim as optim

net = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 64 * 64, 10)
)

criterion = nn.CrossEntropyLoss()

Initialize the optimizer for training the model.

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

Set up a learning rate scheduler to adjust the learning rate during training.

In [ ]:
from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer, step_size=10, gamma=0.1)